In [1]:
import os
import numpy as np
from datetime import date
from gapi.utils.drive_io import list_files, get_google_sheet, gsheet2df, uploadFile
from attendance import get_raw_daily_activities, get_events4attendance, get_first_15,\
get_full_overlap, get_students
from helpers import create_daily_attendance_folder


from configs import room_id2name, BDS021_calendar_name2id, \
dual_track, input_folder, student_list_sheetid, names, tmp_folder_for_attendance, share_lst

cred_path = "./creds"

In [2]:
date_to_aggregate = '2020-04-06'
# str(date.today())

In [3]:
date_to_aggregate

'2020-04-06'

### Create Folder to upload csv files

In [4]:
from attendance import upload_non_duplicated_file
from helpers import get_existing_remote_attn_shts

uploaded, activities_folder_today = get_existing_remote_attn_shts(cred_path, input_folder, date_to_aggregate, share_lst)


from datetime import datetime
local_files_folder = tmp_folder_for_attendance


upload_non_duplicated_file(
    cred_path, local_files_folder, uploaded, date_to_aggregate,
    tmp_folder_for_attendance, activities_folder_today)


Permission Id: 11451666384490214826
Permission Id: 00550734391480679120
File ID: 1khHAXJeadUCfJJT8ndU4z43DBntN4Trr
File ID: 1B3tO45RVVAZciEmVO4aPVCWzpal9nDkr


### Collect Data

In [5]:
# Events today
activities_df = get_raw_daily_activities(cred_path, activities_folder_today, room_id2name)
activities_df["Name"] = activities_df["Name"].apply(lambda s: names[s.lower()] if s.lower() in names else s.lower())

events_today = get_events4attendance(cred_path, BDS021_calendar_name2id, date_to_aggregate, dual_track)

# Students list
df_students = get_students(cred_path, student_list_sheetid)


In [6]:
mg = activities_df.merge(events_today, on="Room")
mg = get_first_15(mg).dropna().reset_index(drop=True)
tmp = mg[["Name", "session", "joined"]].groupby(
    ["Name", "session"]
).agg({"joined": lambda x: len(x)>0})
attendance = df_students.merge(tmp, on=["Name", "session"], how="left")


attendance = attendance.groupby(
    ["Name", "session"]
).agg(
    {"joined":lambda x: np.sum(x)>0}
).sort_values(["Name", "session"], ascending=False).reset_index()
attendance = attendance.merge(df_students, on=["Name", "session"])[["Room", "Name", "session", "joined"]]
attendance["Name"] = attendance["Name"].str.title()
attendance = attendance.sort_values(
    ["Room", "Name", "session"], ascending=[True, True, False]).reset_index(drop=True)
attendance

,Room,Name,session,joined
0,Room 1,Alexander Tin,morning,True
1,Room 1,Alexander Tin,afternoon,False
2,Room 1,Anthony Ali,morning,True
3,Room 1,Anthony Ali,afternoon,False
4,Room 1,Catherine Tang Kim Sin,morning,True
...,...,...,...,...
71,Room 2,Suborna Nath,afternoon,True
72,Room 2,Sundus Seif,morning,True
73,Room 2,Sundus Seif,afternoon,True
74,Room 2,Zack Zbar,morning,True


In [7]:
# attendance.to_csv("./attendance.csv")

In [8]:
mg = activities_df.merge(events_today, on="Room")
mg = get_full_overlap(mg).dropna().reset_index(drop=True)


mg["classtime"] = mg["end"] - mg["start"]
mg["engagedtime"] = mg["left"] - mg["joined"]
aggregated = mg[["Name", "summary", "classtime", "engagedtime"]].groupby(["Name", "summary"]).agg({
    "classtime":"first", "engagedtime":np.sum
}).assign(activities_percentage=lambda x:x["engagedtime"]/x["classtime"])
aggregated

classtime  \
Name          summary                                                
aiko liu      Program Orientation & Introductions [Alex]  01:00:00   
              TA Introduction & Tuition Collection [TA]   01:30:00   
alex baransky Program Orientation & Introductions [Alex]  01:00:00   
              TA Introduction & Tuition Collection [TA]   01:30:00   
              [DSTK] Docker Quick Start [Thomas]          02:30:00   
...                                                            ...   
zack zbar     Program Orientation & Introductions [Alex]  01:00:00   
              TA Introduction & Tuition Collection [TA]   01:30:00   
              [DSTK] Docker Quick Start [Thomas]          02:30:00   
zeyu zhang    Program Orientation & Introductions [Alex]  01:00:00   
              TA Introduction & Tuition Collection [TA]   01:30:00   

                                                         engagedtime  \
Name          summary                                                  
aiko liu      Program Orientation & Introductions [Alex]    00:59:01   
              TA Introduction & Tuition Collection [TA]     00:52:55   
alex baransky Program Orientation & Introductions [Alex]    01:00:00   
              TA Introduction & Tuition Collection [TA]     01:04:19   
              [DSTK] Docker Quick Start [Thomas]            02:08:34   
...                                                              ...   
zack zbar     Program Orientation & Introductions [Alex]    01:00:00   
              TA Introduction & Tuition Collection [TA]     01:01:32   
              [DSTK] Docker Quick Start [Thomas]            02:01:36   
zeyu zhang    Program Orientation & Introductions [Alex]    01:00:00   
              TA Introduction & Tuition Collection [TA]     00:33:45   

                                                          activities_percentage  
Name          summary                                                            
aiko liu      Program Orientation & Introductions [Alex]               0.983611  
              TA Introduction & Tuition Collection [TA]                0.587963  
alex baransky Program Orientation & Introductions [Alex]               1.000000  
              TA Introduction & Tuition Collection [TA]                0.714630  
              [DSTK] Docker Quick Start [Thomas]                       0.857111  
...                                                                         ...  
zack zbar     Program Orientation & Introductions [Alex]               1.000000  
              TA Introduction & Tuition Collection [TA]                0.683704  
              [DSTK] Docker Quick Start [Thomas]                       0.810667  
zeyu zhang    Program Orientation & Introductions [Alex]               1.000000  
              TA Introduction & Tuition Collection [TA]                0.375000  

[136 rows x 3 columns]